In [1]:
%matplotlib inline
%load_ext autoreload
# %load_ext toc2
%autoreload 2

# pymo Dependencies

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pickle
import itertools
from datetime import datetime


import glob
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import IPython

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import *
from pymo.writers import *

In [7]:
# PyRep Libraries
from pyrep import PyRep
from pyrep.objects.shape import Shape
from pyrep.objects.dummy import Dummy
from pyrep.objects.vision_sensor import VisionSensor
from pyrep.const import PrimitiveShape
from pyrep.const import TextureMappingMode
import time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [25]:
p = BVHParser()

# MoCAP Data
# data_all = [p.parse('hello_wave_141_16.bvh')]
data_all = [p.parse('AV_8Walk_Meredith_HVHA_Rep1.bvh')]

In [26]:
BVH2Pos = MocapParameterizer('position')
data_pos = BVH2Pos.fit_transform(data_all)

In [5]:
pr = PyRep()
# Launch the application with a scene file in non-headless mode

pr.launch('scene.ttt', headless=False) 

In [38]:
pr.start()  # Start the simulation

In [37]:
skeletonJoint_dummyHandle = {}
for i in data_all[0].traverse():
    if 'Nub' in i:
        continue 
    skeletonJoint_dummyHandle[i] = Dummy.create(size=0.04)

In [40]:
# Dummy for playing back mocap
scale = 0.015
translation = [0.0, 0.0, 0]

for frame_orient, frame_pos in zip(data_all[0].values.iterrows(), data_pos[0].values.iterrows()):
    
    for skeletonJoint, dummyHandle in skeletonJoint_dummyHandle.items():
        
        # Extract Orientation and Position from MoCAP
        orient = frame_orient[1][skeletonJoint+'_Xrotation'], frame_orient[1][skeletonJoint+'_Yrotation'], frame_orient[1][skeletonJoint+'_Zrotation']
        
        pos = (-frame_pos[1][skeletonJoint+'_Zposition']*scale)+translation[0], (-frame_pos[1][skeletonJoint+'_Xposition']*scale)+translation[1], (frame_pos[1][skeletonJoint+'_Yposition']*scale)
        
        # Set Dummy Position and Orientation
        dummyHandle.set_orientation(np.deg2rad(orient)) 
        dummyHandle.set_position(pos)
        
    pr.step() # Step physics simulation

In [41]:
pr.stop()  # Stop the simulation

In [ ]:
pr.shutdown()  # 